In [1]:
# !wget 'https://anaconda.org/conda-forge/gdcm/2.8.9/download/linux-64/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -q
# !conda install 'gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
# !wget 'https://anaconda.org/conda-forge/libjpeg-turbo/2.1.0/download/linux-64/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -q
# !conda install 'libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y

In [2]:
!pip install python-gdcm
# !pip install pylibjpeg
# !pip install pylibjpeg-libjpeg

     |████████████████████████████████| 9.4 MB 1.2 MB/s 


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from pathlib import Path
import os
from tqdm import tqdm
import re
import gdcm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

!ls ../input

siim-covid19-detection


In [4]:
train_image_level = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
cols = train_image_level.columns.tolist()
cols.insert(0, cols.pop())  # Move StudyInstanceUID to front
train_image_level = train_image_level[cols]
train_image_level.head()


,StudyInstanceUID,id,boxes,label
0,5776db0cec75,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....
1,ff0879eb20ed,000c3a3f293f_image,NaN,none 1 0 0 1 1
2,9d514ce429a7,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....
3,28dddc8559b2,001398f4ff4f_image,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331
4,dfd9fdd85a3e,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...


In [5]:
train_study_level = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
train_study_level['Result'] = (train_study_level['Negative for Pneumonia'].astype(str)
          + train_study_level['Typical Appearance'].astype(str)
          + train_study_level['Indeterminate Appearance'].astype(str)
          + train_study_level['Atypical Appearance'].astype(str))
train_study_level['StudyInstanceUID'] = train_study_level['id'].apply(lambda x: x.replace('_study', ''))
del train_study_level['id']
cols = train_study_level.columns.tolist()
cols.insert(0, cols.pop())  # Move StudyInstanceUID to front
train_study_level = train_study_level[cols]
train_study_level.head()

,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,Result
0,00086460a852,0,1,0,0,0100
1,000c9c05fd14,0,0,0,1,0001
2,00292f8c37bd,1,0,0,0,1000
3,005057b3f880,1,0,0,0,1000
4,0051d9b12e72,0,0,0,1,0001


In [6]:
# Study distribution before train-dev split
study_dist = (train_study_level[['Result', 'StudyInstanceUID']]
            .groupby(['Result']).size().reset_index(name='Count'))
study_dist['Percent'] = study_dist.Count / sum(study_dist.Count)
study_dist

,Result,Count,Percent
0,0001,474,0.078295
1,0010,1049,0.173274
2,0100,2855,0.471589
3,1000,1676,0.276842


In [7]:
study_image_count = (train_image_level.merge(train_study_level, on='StudyInstanceUID')
                     .groupby('StudyInstanceUID').size().reset_index(name='ImageCount'))
study_image_count['Group'] = study_image_count.ImageCount.astype(str)
study_image_count.loc[study_image_count.ImageCount >= 3, 'Group'] = '3+'
del study_image_count['ImageCount']
study_image_count.head()

,StudyInstanceUID,Group
0,00086460a852,1
1,000c9c05fd14,1
2,00292f8c37bd,1
3,005057b3f880,1
4,0051d9b12e72,1


In [8]:
train_study_level = train_study_level.merge(study_image_count, on='StudyInstanceUID')
train_study_level.head()

,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,Result,Group
0,00086460a852,0,1,0,0,0100,1
1,000c9c05fd14,0,0,0,1,0001,1
2,00292f8c37bd,1,0,0,0,1000,1
3,005057b3f880,1,0,0,0,1000,1
4,0051d9b12e72,0,0,0,1,0001,1


In [9]:
study_group_summary = study_image_count.groupby('Group').size().reset_index(name='Count')
study_group_summary['Percent'] = study_group_summary.Count / sum(study_group_summary.Count)
study_group_summary

,Group,Count,Percent
0,1,5822,0.961678
1,2,207,0.034192
2,3+,25,0.004130


In [10]:
# Set aside 'dev' data
study_group = train_study_level.Group
train_study, dev_study, train_group, dev_group = train_test_split(train_study_level, study_group,
                                                                  train_size=0.8, random_state=123,
                                                                  stratify=study_group)

In [11]:
train_study_group_summary = train_study.groupby('Group').size().reset_index(name='Count')
train_study_group_summary['Percent'] = train_study_group_summary.Count / sum(train_study_group_summary.Count)
train_study_group_summary

,Group,Count,Percent
0,1,4657,0.961594
1,2,166,0.034276
2,3+,20,0.004130


In [12]:
dev_study_group_summary = dev_study.groupby('Group').size().reset_index(name='Count')
dev_study_group_summary['Percent'] = dev_study_group_summary.Count / sum(dev_study_group_summary.Count)
dev_study_group_summary

,Group,Count,Percent
0,1,1165,0.962015
1,2,41,0.033856
2,3+,5,0.004129


In [13]:
train_image = train_image_level.merge(train_study, on='StudyInstanceUID')
train_image.head()

,StudyInstanceUID,id,boxes,label,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,Result,Group
0,5776db0cec75,000a312787f2_image,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,0,1,0,0,0100,1
1,ff0879eb20ed,000c3a3f293f_image,NaN,none 1 0 0 1 1,1,0,0,0,1000,1
2,9d514ce429a7,0012ff7358bc_image,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,0,1,0,0,0100,1
3,dfd9fdd85a3e,001bd15d1891_image,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,0,1,0,0,0100,1
4,84543edc24c2,0022227f5adf_image,"[{'x': 1857.2065, 'y': 508.30565, 'width': 376...",opacity 1 1857.2065 508.30565 2233.23384 907.8...,0,0,1,0,0010,1


In [14]:
# From https://www.kaggle.com/xhlulu/siim-covid-19-convert-to-jpg-256px
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from PIL import Image

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data, data.shape

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [15]:
img_size = 1024
train_scaled = train_image.copy()
input_path = Path('../input/siim-covid19-detection')
output_path = Path('../working/png')
output_path.mkdir(parents=True, exist_ok=True)
train_scaled['nx'] = img_size
train_scaled['ny'] = img_size
train_scaled['nx_org'] = 0
train_scaled['ny_org'] = 0
train_scaled['sx'] = 1.0
train_scaled['sy'] = 1.0


for index, row in tqdm(train_scaled.iterrows(), total=len(train_scaled), desc='Rescale images'):
    image_base = re.sub(r'_image$', '', row.id)
    dcm_name = image_base + '.dcm'
    png_name = image_base + '.png'
    study_path = input_path/'train'/row.StudyInstanceUID
    dcm_filepath = next((study_path).rglob('*/' + dcm_name))
    png_path = output_path/row.StudyInstanceUID
    png_path.mkdir(parents=True, exist_ok=True)
    png_filepath = png_path/png_name
    xray, xray_shape = read_xray(dcm_filepath)
    img = resize(xray, size=img_size)
    img.save(png_filepath)
    row.nx_org = xray_shape[0]
    row.ny_org = xray_shape[1]

train_scaled.sx = train_scaled.nx / train_scaled.nx_org
train_scaled.sy = train_scaled.ny / train_scaled.ny_org
compression_opts = dict(method='zip', archive_name='train_scaled.csv')
train_scaled.to_csv('../working/train_scaled.zip', index=False, compression=compression_opts)


Rescale images: 100%|██████████| 5066/5066 [1:02:50<00:00,  1.34it/s]


In [16]:
# !ls ../input/siim-covid19-detection/train/5776db0cec75/81456c9c5423

!ls ../working/png/


00086460a852  323996eaceae  68adf82510a6  99606cd849bc	cb03bff9dfc6
000c9c05fd14  324b025d8c13  68b2e448f6ec  996e865f85d2	cb04434a6c7e
0051d9b12e72  325c6a75b0a5  68be8460c512  997b769bd2bd	cb0d6854e691
00792b5c8852  3271d3fa61e6  68cfbb3d0757  99a33da42b9f	cb4d8159a8bf
00908ffd2d08  328c8b6431a3  68d167b7ed15  99aa2bb52d7e	cb542a3e1086
009bc005edaa  32bb22d329ea  68d1b3e6d222  99ba95380399	cb6618d86e34
00a87235ca36  32bf7e54a7ee  68d7d715efe4  99c38d2b32b3	cb6673ee6433
00b33b3eb8d9  32d20157a012  68d9575beb22  99cb21faf18a	cb8d01dc0d0b
00c241c3fc0d  3300733fc823  68da89574353  99cb5766690f	cb9e68a7ddc6
00c74279c5b7  3309d981e5fe  68ebe7eadfc2  99cf4e630194	cba1253c6b09
00c83e33588f  3312377895b6  68edcbacd401  9a14755ecb62	cba6e524747c
00e936c58da6  33198ca96dce  6909e8b953a5  9a28b92792aa	cba7dd0a06d2
00f9e183938e  33354bc058ae  690a40dcd44c  9a493656953a	cbaee64f9d29
011475cb6db4  3340562b3fce  690cdb1d60c0  9a4a6abdf776	cbb7b764ca87
01206a422293  334ac465b405  690e42810486  9a55a2